In [1]:
import os
import pandas as pd
import re
import numpy as np
import datetime
from catboost import CatBoostRegressor

In [2]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

/tmp/ipykernel_380066/3194276147.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("./data/train.csv")


In [3]:
def days_cyclical_denominator(year, month):
    tmp = int(month + 1)
    return (datetime.datetime(int(year) + (tmp//13),tmp-12 if tmp>=13 else tmp, 1) - datetime.timedelta(days=1)).day

In [4]:
def feature_enginnering(df_train):
    df_train['시'] = df_train['시군구'].map(lambda o: o.split()[0])
    df_train['군구'] = df_train['시군구'].map(lambda o: o.split()[1])
    df_train['동'] = df_train['시군구'].map(lambda o: o.split()[2])
    
    df_train['계약년'] = df_train['계약년월'].map(lambda o: int(str(o)[:4]))
    df_train['계약월'] = df_train['계약년월'].map(lambda o: int(str(o)[4:]))
    df_train['계약월_sin'] = df_train['계약월'].map(lambda o: np.sin((2 * o * np.pi)/12))
    df_train['계약월_cos'] = df_train['계약월'].map(lambda o: np.cos((2 * o * np.pi)/12))
    df_train['계약일_sin'] = df_train.apply(lambda o: np.sin((2*o['계약일']*np.pi)/(days_cyclical_denominator(o['계약년'],o['계약월']))),axis=1)
    df_train['계약일_cos'] = df_train.apply(lambda o: np.cos((2*o['계약일']*np.pi)/(days_cyclical_denominator(o['계약년'],o['계약월']))),axis=1)
    
    df_train = df_train.drop(columns=['계약월','계약일'])
    
    lst = [
        ('k-사용검사일-사용승인일','사용승인일'),
        ('k-등록일자','등록일자'),
        ('k-수정일자','수정일자'),
        ('단지승인일','단지승인일'),
    ]
    for x1,x2 in lst:
        df_train.loc[df_train[x1].isna(),x1] = ""
        df_train[f'{x2} 년'] = df_train[x1].map(lambda o: int(str(o)[:4]) if o!="" else None)
        df_train[f'{x2} 월'] = df_train[x1].map(lambda o: int(str(o)[5:7]) if o!="" else None)
        df_train[f'{x2} 일'] = df_train[x1].map(lambda o: int(str(o)[8:10]) if o!="" else o)
        df_train[f'{x2} 월 sin'] = df_train.apply(lambda o: None if o[x1]=="" else np.sin((2*int(o[x1][5:7]))*np.pi)/12,axis=1)
        df_train[f'{x2} 월 cos'] = df_train.apply(lambda o: None if o[x1]=="" else np.cos((2*int(o[x1][5:7]))*np.pi)/12,axis=1)
        df_train[f'{x2} 일 sin'] = df_train.apply(lambda o: None if o[f'{x2} 일']=="" else np.sin(2*int(o[f'{x2} 일'])*np.pi)/days_cyclical_denominator(o[f'{x2} 년'],o[f'{x2} 월']),axis=1)
        df_train[f'{x2} 일 cos'] = df_train.apply(lambda o: None if o[f'{x2} 일']=="" else np.cos(2*int(o[f'{x2} 일'])*np.pi)/days_cyclical_denominator(o[f'{x2} 년'],o[f'{x2} 월']),axis=1)
    
        df_train = df_train.drop(columns=[x1, f'{x2} 월',f'{x2} 일'])
    
    df_train['거래유형_기타'] = 0
    df_train['거래유형_중고거래'] = 0
    df_train['거래유형_직거래'] = 0
    df_train.loc[df_train['거래유형']=='-','거래유형_기타'] = 1
    df_train.loc[df_train['거래유형']=='중고거래','거래유형_중고거래'] = 1
    df_train.loc[df_train['거래유형']=='직거래','거래유형_직거래'] = 1
    
    df_train = df_train.drop(columns='거래유형')
    
    df_train.loc[df_train['k-단지분류(아파트,주상복합등등)'].isna(),'k-단지분류(아파트,주상복합등등)'] = "기타"
    
    df_tmp = pd.get_dummies(df_train['k-단지분류(아파트,주상복합등등)'],dtype=int)
    # df_tmp.columns = ["단지분류 기타","단지분류 도시형 생활주택(아파트)","단지분류 도시형 생활주택(주상복합)","단지분류 아파트","단지분류 연립주택","단지분류 주상복합"]
    df_tmp.columns = [f"단지분류 {x}" for x in df_tmp.columns.tolist()]
    df_train = pd.concat([df_train, df_tmp], axis=1)
    
    df_train = df_train.drop(columns='k-단지분류(아파트,주상복합등등)')
    
    df_train.loc[df_train['k-세대타입(분양형태)'].isna(),'k-세대타입(분양형태)'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['k-세대타입(분양형태)'],dtype=int)
    # df_tmp.columns = ["분양형태 기타","분양형태 분양","분양형태 알수없음","분양형태 임대"]
    df_tmp.columns = [f"분양형태 {x}" for x in df_tmp.columns.tolist()]
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='k-세대타입(분양형태)')
    
    df_train.loc[df_train['k-관리방식'].isna(),'k-관리방식'] = "기타"
    df_tmp = pd.get_dummies(df_train['k-관리방식'],dtype=int)
    df_tmp.columns = list(map(lambda o: f"관리방식 {o}", df_tmp.columns.tolist()))
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='k-관리방식')
    
    df_train.loc[df_train['k-복도유형'].isna(),'k-복도유형'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['k-복도유형'],dtype=int)
    df_tmp.columns = list(map(lambda o: f"복도유형 {o}",df_tmp.columns.tolist()))
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='k-복도유형')
    
    df_train.loc[df_train['k-난방방식'].isna(),'k-난방방식'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['k-난방방식'],dtype=int)
    df_tmp.columns = list(map(lambda o: f"난방방식 {o}", df_tmp.columns.tolist()))
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='k-난방방식')
    
    df_train.loc[df_train['k-135㎡초과'].isnull(),'k-135㎡초과'] = 0
    df_train.loc[df_train['k-135㎡초과'].notnull(),'k-135㎡초과'] = 1

    """
    df_train.loc[df_train['고용보험관리번호'].notnull(),'고용보험관리번호'] = df_train[df_train['고용보험관리번호'].notnull()]['고용보험관리번호'].map(lambda o: o.replace('-',''))
    df_tmp = pd.get_dummies(df_train['고용보험관리번호'],dtype=int)
    dic_emp_num = {df_tmp.columns[i] : i for i in range(len(df_tmp.columns))}
    df_tmp.columns = [f"고용보험관리번호 {dic_emp_num[x]}" for x in df_tmp.columns]
    df_train = pd.concat([df_train,df_tmp],axis=1)
    """
    
    df_train = df_train.drop(columns='고용보험관리번호')
    
    df_train.loc[df_train['경비비관리형태'].isnull(),'경비비관리형태'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['경비비관리형태'],dtype=int)
    df_tmp.columns = [f'경비비관리형태 {x}' for x in df_tmp.columns.tolist()]
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='경비비관리형태')
    
    df_train.loc[df_train['세대전기계약방법'].isnull(),'세대전기계약방법'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['세대전기계약방법'],dtype=int)
    df_tmp.columns = [f'세대전기계약방법 {x}' for x in df_tmp.columns.tolist()]
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='세대전기계약방법')
    
    df_train.loc[df_train['청소비관리형태'].isnull(),'청소비관리형태'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['청소비관리형태'],dtype=int)
    df_tmp.columns = [f'청소비관리형태 {x}' for x in df_tmp.columns.tolist()]
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='청소비관리형태')
    
    df_train.loc[df_train['기타/의무/임대/임의=1/2/3/4'].isnull(),'기타/의무/임대/임의=1/2/3/4'] = "알수없음"
    df_tmp = pd.get_dummies(df_train['기타/의무/임대/임의=1/2/3/4'],dtype=int)
    df_tmp.columns = [f'기타/의무/임대/임의=1/2/3/4 {x}' for x in df_tmp.columns.tolist()]
    df_train = pd.concat([df_train,df_tmp],axis=1)
    
    df_train = df_train.drop(columns='기타/의무/임대/임의=1/2/3/4')
    
    df_train.loc[df_train['사용허가여부']=='Y','사용허가여부'] = 1
    df_train.loc[df_train['사용허가여부'].isnull(),'사용허가여부'] = 0
    
    df_train['관리비 업로드'] = df_train['관리비 업로드'].map(lambda o: 1 if o=='Y' else 0)
    
    df_train.loc[df_train['사용허가여부']=='Y','사용허가여부'] = 1
    df_train.loc[df_train['사용허가여부']=='N','사용허가여부'] = 0
    
    df_train = df_train.drop(columns=[
        '해제사유발생일',
        '등기신청일자',
        '중개사소재지',
        'k-전화번호',
        'k-팩스번호',
        '단지소개기존clob',
        'k-홈페이지',
        '단지신청일',
        '번지',
        '시군구'
    ])
    
    df_train.loc[:,['시', '군구','동','아파트명','도로명','k-시행사','k-건설사(시공사)']] = df_train[['시', '군구','동','아파트명','도로명','k-시행사','k-건설사(시공사)']].astype('str')
    
    return df_train

In [5]:
df_train = feature_enginnering(df_train)

In [6]:
df_test = feature_enginnering(df_test)

In [7]:
lst = list(filter(lambda o: o not in df_test.columns.to_list(), df_train.columns.to_list()))
for x in lst:
    df_test[x] = 0

In [8]:
df_y = df_train['target']

In [9]:
df_train = df_train.drop(columns=['target'])

In [10]:
model = CatBoostRegressor(
        n_estimators=100, 
        task_type='GPU',
        objective='RMSE',
        thread_count=1,
        bootstrap_type =  "Bernoulli",
        sampling_frequency= 'PerTree',
        verbose=0,
        cat_features=['시', '군구','동','아파트명','도로명','k-시행사','k-건설사(시공사)'],
        leaf_estimation_iterations=1
)

In [11]:
fit_model = model.fit(df_train,df_y)

In [13]:
results = list(model.predict(df_test))

In [20]:
results = list(map(lambda o: round(o), results))

In [23]:
df_results = pd.DataFrame(results)
df_results.columns = ['target']
df_results.to_csv("results.csv",index=False)